In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import numpy as np
import lightgbm as lgb
import pandas as pd
from kaggle.competitions import twosigmanews
import matplotlib.pyplot as plt
import random
from datetime import datetime, date
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
import time

from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
import time

# Any results you write to the current directory are saved as output.

['marketdata_sample.csv', 'news_sample.csv']


In [2]:
env = twosigmanews.make_env()

(market_train_df, news_train_df) = env.get_training_data()
market_train, news_train = market_train_df.copy(), news_train_df.copy()

Loading the data... This could take a minute.
Done!


In [3]:
def populate(data):
    for i in data.columns:
        if (data[i].dtype == "int64" or data[i].dtype == "float64"):
            data[i] = data[i].fillna(data[i].mean())
        elif data[i].dtype == "object":
            data[i] = data[i].fillna("other")
        else:
            pass
        
    return data

market_train_df = populate(market_train_df)

def data_prep(market_dataset):
    market_dataset.time = market_dataset.time.dt.date
    params = {k: v for v, k in enumerate(market_dataset['assetCode'].unique())}
    market_dataset['assetCodeT'] = market_dataset['assetCode'].map(params)
    
    market_dataset = market_dataset.dropna(axis=0)
    
    return market_train

market_train = data_prep(market_train_df)

# check the shape
print(market_train.shape)

(4072956, 17)


In [4]:
market_dataset = market_dataset.loc[market_dataset['time']>=date(2009, 1, 1)]
up = market_train.returnsOpenNextMktres10 >= 0
fcol = [code for code in market_train if code not in ['assetCode', 'assetCodes', 'assetCodesLen', 'assetName', 'audiences', 
                                             'firstCreated', 'headline', 'headlineTag', 'marketCommentary', 'provider', 
                                             'returnsOpenNextMktres10', 'sourceId', 'subjects', 'time', 'time_x', 'universe','sourceTimestamp']]

X = market_dataset[fcol].values
up = up.values
r = market_dataset.returnsOpenNextMktres10.values

# Normalizing
mins = np.min(X, axis=0)
maxs = np.max(X, axis=0)
rng = maxs - mins

X = 1 - ((maxs - X) / rng)

X_train, X_test, up_train, up_test, r_train, r_test = model_selection.train_test_split(X, up, r, test_size=0.25, random_state=99)

train_data = lgb.Dataset(X_train, label=up_train.astype(int))
test_data = lgb.Dataset(X_test, label=up_test.astype(int))

In [8]:
#These were found from seperate file -- see explor.py
x_1 = [0.19000424246380565, 2452, 212, 328, 202]

params_1 = {
        'task': 'train',
        'boosting_type': 'dart',
        'objective': 'binary',
        'learning_rate': x_1[0],
        'num_leaves': x_1[1],
        'min_data_in_leaf': x_1[2],
        'num_iteration': x_1[3],
        'max_bin': x_1[4],
        'verbose': 1
    }


gbm_1 = lgb.train(params_1,
        train_data,
        num_boost_round=100,
        valid_sets=test_data,
        early_stopping_rounds=5)
        
    
#prediction
days = env.get_prediction_days()
n_days = 0
prep_time = 0
prediction_time = 0
packaging_time = 0


for (market_obs_df, news_obs_df, predictions_template_df) in days:
    n_days += 1
    if (n_days%50==0):
        print(n_days,end=' ')
    t = time.time()
    market_obs_df = data_prep(market_obs_df)
    market_obs_df = market_obs_df[market_obs_df.assetCode.isin(predictions_template_df.assetCode)]
    X_live = market_obs_df[fcol].values
    X_live = 1 - ((maxs - X_live) / rng)
    prep_time += time.time() - t
    
    t = time.time()
    lp = (gbm_1.predict(X_live) + gbm_2.predict(X_live))/2
    prediction_time += time.time() -t
    
    t = time.time()

    confidence = lp
    confidence = (confidence-confidence.min())/(confidence.max()-confidence.min())
    confidence = confidence * 2 - 1
    preds = pd.DataFrame({'assetCode':market_obs_df['assetCode'],'confidence':confidence})
    predictions_template_df = predictions_template_df.merge(preds,how='left').drop('confidenceValue',axis=1).fillna(0).rename(columns={'confidence':'confidenceValue'})
    env.predict(predictions_template_df)
    packaging_time += time.time() - t
    
env.write_submission_file()
sub  = pd.read_csv("submission.csv")

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:116: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's binary_logloss: 0.690737
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.689224
[3]	valid_0's binary_logloss: 0.688185
[4]	valid_0's binary_logloss: 0.687514
[5]	valid_0's binary_logloss: 0.687052
[6]	valid_0's binary_logloss: 0.686761
[7]	valid_0's binary_logloss: 0.686554
[8]	valid_0's binary_logloss: 0.686597
[9]	valid_0's binary_logloss: 0.686394
[10]	valid_0's binary_logloss: 0.686249
[11]	valid_0's binary_logloss: 0.686118
[12]	valid_0's binary_logloss: 0.686102
[13]	valid_0's binary_logloss: 0.686038
[14]	valid_0's binary_logloss: 0.685929
[15]	valid_0's binary_logloss: 0.685906
[16]	valid_0's binary_logloss: 0.685812
[17]	valid_0's binary_logloss: 0.685791
[18]	valid_0's binary_logloss: 0.685715
[19]	valid_0's binary_logloss: 0.685565
[20]	valid_0's binary_logloss: 0.685433
[21]	valid_0's binary_logloss: 0.685415
[22]	valid_0's binary_logloss: 0.685392
[23]	valid_0's binary_logloss: 0.685356
[24]	valid_0's binary_loglos

Exception: You can only call `get_prediction_days` once.